In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd 

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 101.1/data/titanic.csv')
#  names = [] serve per sostituire tutti i nomi delle colonne 
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML 101.1/data/car.data", names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class values"])
# skiprows=[0] serve per saltare una o piu row specifiche, in questo caso usato per saltare la prima riga poiche i nomi delle colonne nel file erano nella seconda
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ML 101.1/data/default of credit card clients.xls', skiprows=[0])

In [ ]:
# change the order of the columns: ( per convenienza, utilizzare se il target column e nel mezzo del df)
df = df.reindex(columns=['Age', 'Name'])

#particolarita: 
data = {'Name': ['Alice', 'Bob', 'Charlie'], 
        'Age': [25, 30, 35], 
        'City': ['New York', 'Chicago', 'Los Angeles'], 
        'Gender': ['Female', 'Male', 'Male']}
df = pd.DataFrame(data)
df = df.reindex(columns=['Gender', 'Age', 'City', 'Name'])
# output:
# index  Gender  Age         City     Name
#   0    Female   25     New York    Alice
#   1      Male   30      Chicago      Bob
#   2      Male   35  Los Angeles  Charlie

#######

# eliminare una colonna:
df = df.drop(['ID'], axis = 1) # axis 0 sono rige 1 sono colonne, specificarlo poiche di default è 0, in questo caso se non specificato eliminerebbe tutti i row con un'info('ID') uguale ad esso di qualsiasi colonna

# piu colonne:
df = df.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis = 1)

#######

# eliminare tutti i valori NAN
df = df.dropna()

#se escono problemi potrebbe aiutare: .reset_index(drop=True)

#######

# trasformare tutti i dati in numeri di una colonna
df['Sex'] = (df['Sex'] == 'male').astype(int) # mette male come 1

# se ci fossero piu di 2 tipi da sostituire fare il seguente modo:
# creare una sorta di lista di cio che vogliamo sostituire
gender_map = {'Male': 1, 'Female': 2, 'Non-binary': 3}

# per la sostituzione
df['Gender'] = df['Gender'].replace(gender_map)

# infine convertire il 'Gender' column a int da str
df['Gender'] = df['Gender'].astype(int)


In [ ]:
#per fare il tabulate
from tabulate import tabulate

info = ['Alcohol', 'MA', 'Ash', 'AA', 'Magnesium', 'TP', 'Flavanoids', 'NF', 'PAc', 'CI', 'Hue', 'OD280/OD315', 'Proline']
daScoprire = [12.37, 1.07, 2.1, 18.5, 88, 3.52, 3.75, .24, 1.95, 4.5, 1.04, 2.77, 660] 
print(tabulate([daScoprire], headers=info))

In [ ]:
# visualizzare il size per ogni classe
print(df.groupby('Class').size())

# output: 
# Class
# 1    59
# 2    71
# 3    48

In [ ]:
# per fare il plot
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt


sns.countplot(x="Class", data=df)
plt.title("Diabetes Dataset")
plt.xlabel("Classes")
plt.ylabel("Counts")
plt.show()

In [ ]:
# trasformare in numpy per poter essere trattato dalla macchina
import numpy as np
dfnp = df.to_numpy()

###

# in questo caso i valori indipendenti(x) saranno dalla seconda colonna in poi e il target(y) sono tutti i row della prima colonna
x=dfnp[:,1:]
y=dfnp[:,0]

# i valori indipendenti(x) saranno tutti i row dalla prima alla penultima colonna, mentre valori target(y) idem ma soltanto dell'ultima colonna
x=dfnumpy[:,:-1]
y=dfnumpy[:,-1] 

###

# per visualizzare i primi valori
print(f'y = {x[:10]}')
print(f'y = {y[:10]}')

# per togliere la dotazione scientifica e facilitare la lettura dei dati
np.set_printoptions(suppress=True)

In [ ]:
# per contare il numero di classi e quante ne contiene, es: Counter({1.0: 59, 2.0: 71, 3.0: 48})
import collections
collections.Counter(y)

In [ ]:
# suddivisione dei dati per l'allenamento della macchina per il test di controllo dell'efficacita
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.30, # by default is 75%-25%
                                                    # shuffle is set True by default,
                                                    stratify = y, # per mantenere le proporzioni che ci sono nei dati di partenza
                                                    random_state = 123) # fix random seed for replicability

# **Metodo KNN**

In [ ]:
# importiamo il metodo di classificazione K-NN
from sklearn.neighbors import KNeighborsClassifier

neighbors = KNeighborsClassifier(n_neighbors=5)
# allenamento
neighbors.fit(X_train,y_train)
# previsione
predict = neighbors.predict(X_test)

print(f'dati predetti dal metodo =                {predict[:10]}') # dati predetti che potrebbero essere sbagliati
print(f'dati reali presenti nel dataset di test = {y_test[:10]}') # dati reali presi dal test


In [ ]:
!pip install mlxtend

# valutiamo il nostro metodo:
from mlxtend.plotting import plot_confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
# visualizziamo la confusion matrix
plot_confusion_matrix(confusion_matrix(y_test, predict), cmap=plt.cm.Reds)
plt.show()

In [ ]:
# visualizziamo adesso gli indicatori e la precisione guardando f1-score, piu alto piu accurato 
print(classification_report(y_test, predict))

**Esempio per trovare il K migliore facendo il confronto con accuracy nel classification_report**

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# creare una lista nel quale verra salvato lo score dell'accuratezza di k
accuracy = []
# Will take some time
from sklearn import metrics
# scrivere un for loop che cambi k da 1 a 40
for k in range(1,41):
    neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
    prediction = neigh.predict(X_test)
    accuracy.append(metrics.accuracy_score(y_test, prediction))
    
# plot
plt.figure(figsize=(10,6))
plt.plot(range(1,40),accuracy,color = 'blue',linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('accuracy vs. K Value')
plt.xlabel('K')
plt.ylabel('Accuracy')
print("Maximum accuracy:-",max(accuracy),"at K =",accuracy.index(max(accuracy)) + 1 ) # + 1 perche l'index parte da 0

**Esempio per trovare il K migliore facendo il confronto con weighted_avg nel classification_report**

In [ ]:
from sklearn.metrics import f1_score

# creare una lista nel quale verra salvato lo score del weighted_avg di k
f1s_weighted_avg = []

# Calculating f1 score for K values between 1 and 40
for i in range(1, 41):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    # using average='weighted' to calculate a weighted average for the 4 classes 
    f1s_weighted_avg.append(f1_score(y_test, pred_i, average='weighted'))

#plot
plt.figure(figsize=(12, 6))
plt.plot(range(1, 41), f1s_weighted_avg, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')
print("Maximum accuracy:-",max(f1s_weighted_avg),"at K =",f1s_weighted_avg.index(max(f1s_weighted_avg)) + 1) # + 1 perche l'index parte da 0

**Esempio per trovare il K migliore facendo il confronto con macro_avg nel classification_report**

In [ ]:
# stesso codice di prima ma con un piccolo ritocco:
    f1s_weighted_avg.append(f1_score(y_test, pred_i, average='weighted')) 
# average='weighted' a posto di weighted immettere macro

# **decision tree**

In [ ]:
from sklearn import tree

tree_clf = tree.DecisionTreeClassifier(criterion="gini", # criteri per stabilire come splittare
                                       max_depth=4, # profondità dell'albero per evitare l'overfitting
                                       min_samples_split=30, # dimensione minima del sottogruppo a cui fermarsi (no more split)
                                       max_leaf_nodes=6, # numero dei nodi foglia
                                       min_samples_leaf=4 # numero di campioni per essere una foglia
                                      )
#allenamento
tree_clf.fit(X_train,y_train)

# predizione
predict_dectree = tree_clf.predict(X_test)

# visualizziamo il risultato (solo i primi dieci)
print(f'dati predetti dal metodo decision tree =  {predict_dectree[:10]}') # dati predetti dal metodo
print(f'dati reali presenti nel dataset di test = {y_test[:10]}') # dati effettivi presenti nel test set

In [ ]:
!pip install mlxtend

# valutiamo il nostro metodo:
from mlxtend.plotting import plot_confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
# visualizziamo la confusion matrix
plot_confusion_matrix(confusion_matrix(y_test, predict_dectree), cmap=plt.cm.Reds)# cmap=plt.cm.Reds solo per il colore
plt.show()

In [ ]:
print(classification_report(y_test, predict_dectree))

**Esempio per trovare il depth migliore facendo il confronto con accuracy nel classification_report**

In [ ]:
# creare una lista nel quale verra salvato lo score dell'accuratezza di k
accuracy = []
# Will take some time
from sklearn import metrics
for depth in range(1,40):
  tree_clf = tree.DecisionTreeClassifier(criterion="gini", # criteri per stabilire come splittare
                                       max_depth=depth, # profondità dell'albero per evitare l'overfitting
                                       min_samples_split=30, # dimensione minima del sottogruppo a cui fermarsi (no more split)
                                       max_leaf_nodes=6, # numero dei nodi foglia
                                       min_samples_leaf=4 # numero di campioni per essere una foglia
                                      )
  # prima parte: dobbiamo fare in modo che il metodo impari dalle informazioni a disposizione
  tree_clf.fit(X_train,y_train)

  # seconda parte: vediamo se il metodo ha imparato bene facendogli prevedere i risultati 
  predict_dt = tree_clf.predict(X_test)
  accuracy.append(metrics.accuracy_score(y_test, predict_dt))
    
#plot
plt.figure(figsize=(10,6))
plt.plot(range(1,40),accuracy,color = 'blue',linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('accuracy vs. K Value')
plt.xlabel('K')
plt.ylabel('Accuracy')
print("Maximum accuracy:-",max(accuracy),"at K =",accuracy.index(max(accuracy)) + 1 ) # + 1 perche l'index parte da 0

**Esempio per trovare il depth migliore facendo il confronto con weighted_avg nel classification_report**

In [ ]:
from sklearn.metrics import f1_score

# invece di:
accuracy = []
accuracy.append(metrics.accuracy_score(y_test, predict_dt))
# diventa:
f1s = []
f1s.append(f1_score(y_test, predict_dt, average='weighted'))

**Esempio per trovare il depth migliore facendo il confronto con macro_avg nel classification_report**

In [ ]:
from sklearn.metrics import f1_score
f1s.append(f1_score(y_test, predict_dt, average='macro'))

# **grafico conda**

In [ ]:
# to be run only once, code for the installation of a new library and the related software
!conda update --force conda
!conda install graphviz
!conda install python-graphviz

In [ ]:
#to be run every time. Tells your computer where to look for graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:\\Users\\wtitz\\Anaconda3\\Library\\bin\\graphviz'

In [ ]:
features=list(df.columns.values) # (sono i nomi delle colonne del dataset)
features.remove('Class')
print(features)


import graphviz
dot_data = tree.export_graphviz(tree_clf, out_file=None, 
                     feature_names=features,  
                     class_names=['0','1','2'], # !!!!!!!!!! questo dipende da quanti classi ci sono, o tipi di target?, esempio nel wine ci sono 3 classi per questo cosi, se ci fossero 2 classi sara: class_names=['0','1']
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)
graph.render('Wines')
graph